<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/apo_tin_arxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Check if Spark was downloaded
!ls


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,346 kB]
Fetched 3,682 kB in 3s (1,412 kB/s)
Reading package lists... Done
medical_tc_test.csv	 spark-3.4.1-bin-hadoop3.tgz
me

In [31]:
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java


update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [32]:
!java -version


openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u372-ga~us1-0ubuntu1~20.04-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


In [33]:
!tar xf spark-3.4.1-bin-hadoop3.tgz


In [34]:
!pip install -q findspark


In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


In [36]:
import findspark
findspark.init()


In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [38]:
!pip install pyspark
!pip install spark-nlp


In [39]:
import pandas as pd

# The URLs where the data is located
url_labels = "https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_labels.csv"
url_test = "https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv"
url_train = "https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv"

# Use pandas to load the data
data_labels = pd.read_csv(url_labels)
data_test = pd.read_csv(url_test)
data_train = pd.read_csv(url_train)


In [40]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp

spark = sparknlp.start()


In [41]:
!wget https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
!wget https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv


--2023-07-08 08:34:55--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14254503 (14M) [text/plain]
Saving to: ‘medical_tc_train.csv.1’

medical_tc_train.cs 100%[===================>]  13.59M  --.-KB/s    in 0.08s   

2023-07-08 08:34:55 (170 MB/s) - ‘medical_tc_train.csv.1’ saved [14254503/14254503]

--2023-07-08 08:34:55--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent

In [42]:
train_data = spark.read.option("header", "true").csv("medical_tc_train.csv")
test_data = spark.read.option("header", "true").csv("medical_tc_test.csv")


In [43]:
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1


In [44]:
from sparknlp.annotator import *
from sparknlp.base import *

# Assemble the text to a document
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# Tokenize the document
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize the tokens (remove non-alphabetic characters and convert to lowercase)
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# Remove stopwords
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

# Use WordEmbeddingsModel pretrained on the 'glove_100d' dataset
word_embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("embeddings")

# Use BertEmbeddings pretrained on the 'bert_base_cased' dataset
bert_embeddings = BertEmbeddings.pretrained('bert_base_cased') \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("bert")

# Assemble all of these steps into a pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    word_embeddings,
    bert_embeddings
])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)

# Transform the training and test data
train_data = model.transform(train_data)
test_data = model.transform(test_data)


TypeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data
# Load the data
labels = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_labels.csv", inferSchema=True, header=True)
train = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_train.csv", inferSchema=True, header=True)
test = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_test.csv", inferSchema=True, header=True)


# Show the first few rows of the DataFrame
labels.show()
train.show()
test.show()



+---------------+--------------------+
|condition_label|      condition_name|
+---------------+--------------------+
|              1|           neoplasms|
|              2|digestive system ...|
|              3|nervous system di...|
|              4|cardiovascular di...|
|              5|general pathologi...|
+---------------+--------------------+

+---------------+--------------------+
|condition_label|    medical_abstract|
+---------------+--------------------+
|              5|Tissue changes ar...|
|              1|Neuropeptide Y an...|
|              2|Sexually transmit...|
|              1|Lipolytic factors...|
|              3|Does carotid rest...|
|              3|The shoulder in m...|
|              2|The management of...|
|              4|Pharmacomechanica...|
|              5|Color Doppler dia...|
|              5|Noninvasive diagn...|
|              4|Sodium sensitive ...|
|              1|Imaging bone tumo...|
|              5|Closure of a bron...|
|              1|Utility

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover


In [ ]:
tokenizer = Tokenizer(inputCol="medical_abstract", outputCol="words")
train = tokenizer.transform(train)
test = tokenizer.transform(test)


In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
train = remover.transform(train)
test = remover.transform(test)


In [ ]:
train.show()
test.show()


+---------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|               words|      filtered_words|
+---------------+--------------------+--------------------+--------------------+
|              5|Tissue changes ar...|[tissue, changes,...|[tissue, changes,...|
|              1|Neuropeptide Y an...|[neuropeptide, y,...|[neuropeptide, y,...|
|              2|Sexually transmit...|[sexually, transm...|[sexually, transm...|
|              1|Lipolytic factors...|[lipolytic, facto...|[lipolytic, facto...|
|              3|Does carotid rest...|[does, carotid, r...|[carotid, resteno...|
|              3|The shoulder in m...|[the, shoulder, i...|[shoulder, multip...|
|              2|The management of...|[the, management,...|[management, post...|
|              4|Pharmacomechanica...|[pharmacomechanic...|[pharmacomechanic...|
|              5|Color Doppler dia...|[color, doppler, ...|[color, doppler, ...|
|              5|Noninvasive

In [ ]:
from pyspark.ml.feature import Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
documentDF = train.select("filtered_words")

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="filtered_words", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))


Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
Vector: [-0.2982249114168237,0.11764717956634275,-0.1962141759779152]

Text: [intraatrial, extension, thyroid, cancer:, technique, results, radical, surgical, approach., occlusion, superior, vena, cava, tumor, thrombus, extending, right, atrium, diagnosed, three, patients, follicular, thyroid, cancer., patients, showed, typical, clinical, picture, superior, vena, cava, syndrome., right, parasternal, thoracotomy, performed, preparation, major, vessels., superior, vena, cava, opened, entire, intravascular, tumor, thrombus, removed., cavotomy, closed, directly, two, patients., third, patient, left, brachiocephalic, trunk, resected, reconstructed, vascular, (polytetrafluoroethylene), graft., patient, bone, brain, metastases, occlusion, graft, 3, months, surgery, anticoagulation, stopped., two, patients, clinically, symptom, free, without, local, recurrence, 13, 50, months, surgery., aggressive, surgical, approach, justified, grossly, i

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Text: [sodium-potassium, interaction, hypertension, hypertensive, cardiovascular, disease., epidemiological, evidence, suggests, low, potassium, intake, associated, probability, occurrence, hypertension, stroke., short-term, response, increased, potassium, intake, increased, sodium, excretion, well, increased, potassium, excretion;, short-term, response, increased, sodium, intake, increased, potassium, excretion, well, increased, sodium, excretion., experimental, studies,, increased, amounts, potassium, able, block, noxious, influences, sodium., sodium, potassium, must, concomitantly, considered, investigation, association, either, cations, hypertension, cardiovascular, disease., chloride, ion, also, important, sodium's, effects;, importance, potassium's, effects, extensively, explored.] => 
Vector: [-0.06835398088662407,-0.13134158864527679,-0.1587418762438402]

Text: [relationship, histologic, features,, dna, flow, cytometry,, clinical, behavior, squamous, cell, carcinomas, larynx., 

In [ ]:
!pip install transformers


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

# Predict hidden states features for each layer
with torch.no_grad():
    outputs = model(tokens_tensor)

# Get the embeddings
embeddings = outputs.last_hidden_state

print(embeddings)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[-0.0549,  0.1053, -0.1065,  ..., -0.3551,  0.0686,  0.6506],
         [-0.5759, -0.3650, -0.1383,  ..., -0.6782,  0.2092, -0.1639],
         [-0.1641, -0.5597,  0.0150,  ..., -0.1603, -0.1346,  0.6216],
         ...,
         [ 0.2448,  0.1254,  0.1587,  ..., -0.2749, -0.1163,  0.8809],
         [ 0.0481,  0.4950, -0.2827,  ..., -0.6097, -0.1212,  0.2527],
         [ 0.9046,  0.2137, -0.5897,  ...,  0.3040, -0.6172, -0.1950]]])


In [ ]:
!pip install spark-nlp==3.4.1


In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

# Start Spark Session with Spark NLP
spark = sparknlp.start()

# Define Spark NLP pipeline
document_assembler = DocumentAssembler()\
    .setInputCol("filtered_words")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

bert_embeddings = BertEmbeddings\
    .pretrained('bert_base_cased', 'en')\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

embeddings_finisher = EmbeddingsFinisher()\
    .setInputCols(["embeddings"])\
    .setOutputCols(["finished_embeddings"])\
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    bert_embeddings,
    embeddings_finisher
])

# Fit the pipeline to your data
model = pipeline.fit(train)

# Transform your data
result = model.transform(train)


TypeError: ignored